# Install Packages

In [ ]:
# Install WhisperX
! pip install git+https://github.com/m-bain/whisperx.git


In [ ]:
!pip install dlib

In [ ]:
!pip install moviepy

In [4]:
import os
# Get the list of all files and directories
# Grayscale - REAL 0 1 | FAKE 
path = "/kaggle/input/deepfake-detection/REAL/REAL/dfdc_train_part_2"
dataset = os.listdir(path)

In [5]:
len(dataset)

230

In [6]:
import pandas as pd

metadata = pd.read_csv("/kaggle/input/deepfake-detection/selected_video.csv")

# Preprocessing


## Import Packages

In [8]:
import librosa
import moviepy.editor as mp
import tempfile
import numpy as np

import cv2
import dlib

import whisperx
import gc

2024-07-07 06:03:22.812517: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 06:03:22.812635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 06:03:22.983813: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Word Alignment Processing


In [ ]:
device = "cuda" 
batch_size = 16
compute_type = "float16"

# Speech to text transcription model
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

def getWordAlignment(audio_file):
    audio = whisperx.load_audio(audio_file)
    result = model.transcribe(audio, batch_size=batch_size)

    # Word alignment model
    word_model, metadata = whisperx.load_align_model(language_code = result["language"], device = device)

    result = whisperx.align(result["segments"],
                        word_model,
                        metadata,
                        audio,
                        device,
                        return_char_alignments = False)

    print(result["segments"])
    return result

In [10]:
# Get start and end timestamp for each word in a MP4 file
def getTimestamp(result):
    word_timestamp = []

    for item in result["segments"]:
        for word in item.get("words"):
            start_end_timestamp = [word.get("start"),word.get("end")]
            word_timestamp.append(start_end_timestamp)

    return word_timestamp

## Video Processing


In [11]:
face_detector = dlib.get_frontal_face_detector()

dlib_facelandmark = dlib.shape_predictor("/kaggle/input/deepfake-detection/shape_predictor_68_face_landmarks.dat")

# Define fixed size for the mouth ROI
fixed_width = 100
fixed_height = 50

def mouth_extractor(cap, start_timestamp, end_timestamp, fps):
    # Get frame rate
    start_frame = int(start_timestamp * fps)
    end_frame = int(end_timestamp * fps)

    # Set the starting frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    processed_frame = [] # final processed frame

    missing_frame_thres = (end_frame - start_frame) / 2 # threshold for termination based on missing frame

    missing_frame = 0 # number of missing frame

    first_frame = True
    num_face_registered = 0 # number of face detected across frame

    all_face = {} # Save all extracted mouth region for each person in the video
    mouth_open_close_diff = {} # Save the difference between mouth top (y) and mouth bottom (y) to determine which person is speaking

    for frame_number in range(start_frame, end_frame+1):
        ret, frame = cap.read()

        if not ret:
            print("Failed to read frame from video.")
            return []

        if (missing_frame >= missing_frame_thres):
            print("Too many missing frame at this section")
            return []

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_detector(gray)

        num_face_detected = len(faces)

        # Check if no faces were detected
        if num_face_detected == 0:
            missing_frame = missing_frame + 1
            continue

        # detected new face in the following frame (after first frame), register new mouth x coordinate
        if (num_face_detected > num_face_registered) and not first_frame:

            # repeat the process by the number of new face detected
            for i in range(num_face_detected - num_face_registered):
                mouth_distance = {}
                
                for face in faces:
                    # get face landmarks
                    landmarks = dlib_facelandmark(gray, face)

                    # extract mouth landmark at 48 as ID
                    mouth_x = landmarks.part(48).x - 15
                    
                    # Find the nearest mouth in the registered face
                    nearest_distance = [0,100000]
                    for registered_mouth in all_face:
                        # Find the distance between two mouth in x coordinate
                        difference = abs(registered_mouth - mouth_x)
                        
                        # Identify the nearest mouth
                        if (difference < nearest_distance[1]):
                            nearest_distance = [mouth_x, difference]
                    
                    mouth_distance[nearest_distance[0]] = nearest_distance[1]
                
                farthest = [0,0]
                # Find the mouth with the largest difference in distance
                for key, distance in mouth_distance.items():
                    if distance > farthest[1]:
                        farthest[0] = key
                        farthest[1] = distance
                
                # register new mouth with empty list
                all_face[farthest[0]] = [] # mouth region
                mouth_open_close_series[farthest[0]] = 0 # difference between top lip (y) and bottom lip (y)

            # update number of face registered
            num_face_registered = num_face_detected


        for face in faces:
            # get face landmarks
            landmarks = dlib_facelandmark(gray, face)

            # extract mouth region
            mouth_x = landmarks.part(48).x - 15
            mouth_y = landmarks.part(51).y - 15
            mouth_w = landmarks.part(54).x - mouth_x + 15
            mouth_h = landmarks.part(57).y - mouth_y + 15


            # When first frame, register the mouth x coordinate to the dictionary
            if first_frame:
                num_face_registered = num_face_detected
                # mouth region
                all_face[mouth_x] = [gray[mouth_y:mouth_y + mouth_h, mouth_x:mouth_x + mouth_w]] 
                
                # difference between top lip (y) and bottom lip (y)
                mouth_open_close_diff[mouth_x] = abs(landmarks.part(62).y - landmarks.part(66).y) 

            else:
                # Append to the nearest mouth_x list
                nearest_x = [99999,0]

                for mouth in all_face:
                    difference = abs(mouth - mouth_x)
                    if (difference < nearest_x[0]):
                        nearest_x[0] = difference # difference between two mouth x coordinate
                        nearest_x[1] = mouth # key

                # Append the frame to the nearest mouth (key)
                all_face[nearest_x[1]].append(gray[mouth_y:mouth_y + mouth_h, mouth_x:mouth_x + mouth_w]) # mouth region
                
                # add the difference between top lip (y) and bottom lip (y)
                mouth_open_close_diff[nearest_x[1]] = mouth_open_close_diff[nearest_x[1]] + abs(landmarks.part(62).y - landmarks.part(66).y)                                          


                # update the mouth x coordinate (key) with the latest position
                all_face[mouth_x] = all_face.pop(nearest_x[1])
                mouth_open_close_diff[mouth_x] = mouth_open_close_diff.pop(nearest_x[1])

        first_frame = False
    
    # Determine the current speaker by the difference between mouth top and bottom position across frame
    # The speaker will have the largest the difference across frame because they will open and close their mouth
    speaker_key = max(mouth_open_close_diff, key=mouth_open_close_diff.get)

    # check number of frame. Return empty list when too many missing frame
    if len(all_face[speaker_key]) <= missing_frame_thres:
        return []

    # Resize frame
    for mouth_roi in all_face[speaker_key]:
        if mouth_roi.shape[1] < fixed_width or mouth_roi.shape[0] < fixed_height:
            interpolation_set = cv2.INTER_CUBIC # For upsampling
        else:
            interpolation_set = cv2.INTER_AREA # For downsampling

        resized_mouth_roi = cv2.resize(mouth_roi, (fixed_width, fixed_height), interpolation=interpolation_set)

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_mouth_roi / 255

        processed_frame.append(normalized_frame)

    return processed_frame



## Audio Processing


In [12]:
def generate_spectrogram(audio_signal, sampling_rate, start_time, end_time):
    
    # Extract the desired clip
    clip = audio_signal[int(start_time*sampling_rate):int(end_time*sampling_rate)]
    
    # Compute the spectrogram
    Mel_spectrogram = librosa.feature.melspectrogram(y=clip)
    
    log_spectrogram = librosa.power_to_db(Mel_spectrogram, ref=np.max)

        
    return log_spectrogram


## Start preprocessing

In [13]:
# Create folder to output processed data
import os

folder_path = '/kaggle/working/'

for item in ["REAL","FAKE"]:
    temp_path = folder_path+item
    
    if not os.path.exists(temp_path):
        os.mkdir(temp_path)


In [ ]:
video_counter = 0
total_video = len(dataset)

for file in dataset:    
    video_counter = video_counter + 1
    print("Processing File: ", file, f"  |   Progress---> {video_counter}/{total_video}")
    video_path = f"{path}/{file}"
e
    # load the video file
    video = mp.VideoFileClip(video_path)

    audio = video.audio

    # Save the audio as a temporary WAV file
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmp_audio_file:
        audio.write_audiofile(tmp_audio_file.name)
        tmp_audio = tmp_audio_file.name

        # get word alignment result
        alignment_result = getWordAlignment(tmp_audio)
        
        # Load the audio using librosa
        y, sr = librosa.load(tmp_audio)
        
    # Get start and end timestamp for each word in the audio file
    word_timestamp = getTimestamp(alignment_result)

    for timestamp in word_timestamp:
        # ensure the timestamp is not empty
        if (timestamp[0] == None ):
            continue
            
        print(f"Timestamp -> {timestamp[0]} - {timestamp[1]}")
        # Read video MP4 File
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        print("FPS: ", fps)

        # timestamp[0] -> start time | timestamp[1] -> end time
        processed_video = mouth_extractor(cap, timestamp[0], timestamp[1], fps)

        # Process and convert audio data to log mel spectrogram
        processed_audio = generate_spectrogram(y, sr, timestamp[0], timestamp[1])
        print('Shape: ', processed_audio.shape)
        
        if not processed_video:
            print("empty frame")
            continue

        # obtain the filename
        filename = os.path.splitext(file)[0]

        # Determine if the MP4 is real or fake
        label = metadata[metadata["file_name"] == file]["label"].values[0]
        
        # Create new folder for the data
        folder_path = f"/kaggle/working/{label}/{filename}_{timestamp[0]}_{timestamp[1]}/"
        os.mkdir(folder_path)
        
        np.save(folder_path+"video", processed_video)
        np.save(folder_path+"audio", processed_audio)

        print("="*100)
        print(f"Saved Data")
        print("="*100)

    cap.release()


Processing File:  jjycletzyl.mp4   |   Progress---> 1/230
MoviePy - Writing audio in /tmp/tmpm3a06xx3.wav


MoviePy - Done.
Detected language: en (0.99) in first 30s of audio...


Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:01<00:00, 259MB/s]  


[{'start': 0.049, 'end': 7.041, 'text': ' So, the movie did a good job of showing what happened, why it happened, and his role in it and that they were freed and how they were freed.', 'words': [{'word': 'So,', 'start': 0.049, 'end': 0.089, 'score': 0.012}, {'word': 'the', 'start': 0.65, 'end': 0.75, 'score': 0.903}, {'word': 'movie', 'start': 0.77, 'end': 0.991, 'score': 0.565}, {'word': 'did', 'start': 1.011, 'end': 1.111, 'score': 0.705}, {'word': 'a', 'start': 1.131, 'end': 1.151, 'score': 0.582}, {'word': 'good', 'start': 1.191, 'end': 1.311, 'score': 0.916}, {'word': 'job', 'start': 1.331, 'end': 1.572, 'score': 0.872}, {'word': 'of', 'start': 1.612, 'end': 1.652, 'score': 0.759}, {'word': 'showing', 'start': 1.692, 'end': 1.932, 'score': 0.872}, {'word': 'what', 'start': 1.972, 'end': 2.113, 'score': 0.867}, {'word': 'happened,', 'start': 2.173, 'end': 2.573, 'score': 0.882}, {'word': 'why', 'start': 2.674, 'end': 2.914, 'score': 0.764}, {'word': 'it', 'start': 2.974, 'end': 3.0

MoviePy - Done.
Detected language: en (0.98) in first 30s of audio...
[{'start': 5.353, 'end': 9.323, 'text': ' something that I think we should dwell on but I do feel that', 'words': [{'word': 'something', 'start': 5.353, 'end': 5.634, 'score': 0.542}, {'word': 'that', 'start': 5.674, 'end': 5.834, 'score': 0.926}, {'word': 'I', 'start': 5.914, 'end': 5.975, 'score': 0.977}, {'word': 'think', 'start': 6.035, 'end': 6.255, 'score': 0.888}, {'word': 'we', 'start': 6.275, 'end': 6.336, 'score': 0.082}, {'word': 'should', 'start': 6.396, 'end': 6.636, 'score': 0.741}, {'word': 'dwell', 'start': 6.737, 'end': 7.097, 'score': 0.722}, {'word': 'on', 'start': 7.338, 'end': 7.458, 'score': 0.936}, {'word': 'but', 'start': 7.559, 'end': 7.779, 'score': 0.871}, {'word': 'I', 'start': 7.92, 'end': 8.02, 'score': 0.943}, {'word': 'do', 'start': 8.16, 'end': 8.381, 'score': 0.996}, {'word': 'feel', 'start': 8.501, 'end': 8.802, 'score': 0.992}, {'word': 'that', 'start': 8.822, 'end': 9.323, 'score'

...

MoviePy - Done.
Detected language: en (0.99) in first 30s of audio...
[{'start': 1.993, 'end': 2.615, 'text': ' Natural.', 'words': [{'word': 'Natural.', 'start': 1.993, 'end': 2.615, 'score': 0.921}]}, {'start': 3.777, 'end': 4.759, 'text': 'Surprised.', 'words': [{'word': 'Surprised.', 'start': 3.777, 'end': 4.759, 'score': 0.912}]}, {'start': 7.105, 'end': 7.786, 'text': 'Natural.', 'words': [{'word': 'Natural.', 'start': 7.105, 'end': 7.786, 'score': 0.837}]}, {'start': 9.309, 'end': 9.911, 'text': 'Open mouth.', 'words': [{'word': 'Open', 'start': 9.309, 'end': 9.59, 'score': 0.942}, {'word': 'mouth.', 'start': 9.65, 'end': 9.911, 'score': 0.928}]}]
Timestamp -> 1.993 - 2.615
FPS:  29.97
Shape:  (128, 27)
Saved Data
Timestamp -> 3.777 - 4.759
FPS:  29.97
Shape:  (128, 43)
Saved Data
Timestamp -> 7.105 - 7.786
FPS:  29.97
Shape:  (128, 30)
Saved Data
Timestamp -> 9.309 - 9.59
FPS:  29.97
Shape:  (128, 13)
Saved Data
Timestamp -> 9.65 - 9.911
FPS:  29.97
Shape:  (128, 12)
Saved Data

MoviePy - Done.


Detected language: en (0.98) in first 30s of audio...
[{'start': 0.009, 'end': 10.026, 'text': ' Caucasian and African American so you do see on the things that they go through and the things that they experience in in America I do believe that we should shed light', 'words': [{'word': 'Caucasian', 'start': 0.009, 'end': 0.57, 'score': 0.728}, {'word': 'and', 'start': 0.59, 'end': 0.67, 'score': 0.166}, {'word': 'African', 'start': 0.79, 'end': 1.131, 'score': 0.593}, {'word': 'American', 'start': 1.151, 'end': 1.612, 'score': 0.912}, {'word': 'so', 'start': 1.652, 'end': 1.832, 'score': 0.94}, {'word': 'you', 'start': 1.872, 'end': 2.012, 'score': 0.806}, {'word': 'do', 'start': 2.032, 'end': 2.273, 'score': 0.845}, {'word': 'see', 'start': 2.393, 'end': 2.674, 'score': 0.782}, {'word': 'on', 'start': 3.054, 'end': 3.134, 'score': 0.983}, {'word': 'the', 'start': 3.174, 'end': 3.275, 'score': 0.986}, {'word': 'things', 'start': 3.335, 'end': 3.595, 'score': 0.929}, {'word': 'that', 's

MoviePy - Done.


Detected language: en (0.98) in first 30s of audio...
[{'start': 1.572, 'end': 9.866, 'text': ' I just feel like bad diets have become a marketing gimmick because people nowadays with technology I feel that plays a huge part of it.', 'words': [{'word': 'I', 'start': 1.572, 'end': 1.732, 'score': 0.213}, {'word': 'just', 'start': 1.772, 'end': 1.972, 'score': 0.57}, {'word': 'feel', 'start': 1.992, 'end': 3.295, 'score': 0.428}, {'word': 'like', 'start': 3.335, 'end': 3.495, 'score': 0.88}, {'word': 'bad', 'start': 3.595, 'end': 3.795, 'score': 0.693}, {'word': 'diets', 'start': 3.835, 'end': 4.076, 'score': 0.49}, {'word': 'have', 'start': 4.096, 'end': 4.216, 'score': 0.271}, {'word': 'become', 'start': 4.256, 'end': 4.517, 'score': 0.826}, {'word': 'a', 'start': 4.537, 'end': 4.557, 'score': 0.971}, {'word': 'marketing', 'start': 4.617, 'end': 4.997, 'score': 0.837}, {'word': 'gimmick', 'start': 5.038, 'end': 5.438, 'score': 0.782}, {'word': 'because', 'start': 5.619, 'end': 5.959, '

## Zip folder to download

In [ ]:
# !zip -r FAKE_02.zip /kaggle/working/FAKE

In [ ]:
import shutil

def remove_directory_and_contents(directory_path):
    try:
        shutil.rmtree(directory_path)
        print(f"Successfully removed the directory and all its contents: {directory_path}")
    except Exception as e:
        print(f"Failed to remove the directory {directory_path}. Reason: {e}")

# Example usage
directory_path = "/kaggle/working/"
remove_directory_and_contents(directory_path)

In [ ]:
!zip -r REAL_GS_02.zip /kaggle/working/REAL